In [1]:
import pandas as pd

In [29]:
# Load in matches and wrestlers dataframes
MATCHES = pd.read_csv('MATCHES.csv').drop(['Unnamed: 0'],axis=1)
WRESTLERS = pd.read_csv('WRESTLERS.csv').drop(['Unnamed: 0'],axis=1)

In [30]:
MATCHES.shape # number of matches, recorded match variables

(14114, 19)

In [31]:
MATCHES.columns

Index(['Weight Class', 'Victory Type (L)', 'Victory Type (S)', 'Winner Score',
       'Loser Score', 'Winner First Name', 'Winner Last Name',
       'Winner School (L)', 'Winner School (S)', 'Loser First Name',
       'Loser Last Name', 'Loser School (L)', 'Loser School (S)',
       'Unknown (Seed?)', 'Round', 'Event Name', 'Event ID',
       'Winner Full Name', 'Loser Full Name'],
      dtype='object')

In [32]:
WRESTLERS.shape # number of wrestlers, recorded wrestler variables

(3915, 9)

In [33]:
WRESTLERS.columns

Index(['First Name', 'Last Name', 'Full Name', 'School Name', 'School Code',
       'Weight Class', 'Wins', 'Losses', 'Matches'],
      dtype='object')

In [34]:
WRESTLERS

,First Name,Last Name,Full Name,School Name,School Code,Weight Class,Wins,Losses,Matches
0,Jose,Ortiz,Jose Ortiz,Inwood Academy HS-PSAL,INW,106,0,2,2
1,Donnasis,George,Donnasis George,Xavier,XAVI,182,6,6,12
2,Grant,Johnston,Grant Johnston,Canandaigua Academy,CAAC,160,0,7,7
3,Chris,Powell,Chris Powell,Centereach,CENT,285,4,9,13
4,Marlon,Quintanilla,Marlon Quintanilla,West Babylon Sr HS,WBSH,195,0,3,3
...,...,...,...,...,...,...,...,...,...
3910,Darryl,Harris,Darryl Harris,Grand Street Campus-PSAL,GSCP,195,5,6,11
3911,Jayshon,Hines,Jayshon Hines,North Babylon Sr High Sch,NBSH,220,7,2,9
3912,Alex,Kowalski,Alex Kowalski,Frontier Sr,FRSR,126,3,7,10
3913,Oscar,Contreras,Oscar Contreras,Lafayette Campus - PSAL,LAF,132,4,0,4
